# Revised case normalization for Hirslanden Linde 2018

Notes:

- inserted 10 cases into the DB



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group, format_for_grouper_one_case, format_for_grouper
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Linde', '2018', 'Änderungen_LI_2018')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Linde', year='2018', sheet='Änderungen_LI_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-14 09:37:00.466 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 10 cases for Hirslanden Linde 2018
2022-11-14 09:37:00.471 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:63 - TYPES:
datum                             string
tranche                           string
case_id                           object
patient_id                        object
kkik                              string
gender                            string
age_years                          int64
bfs_code                          string
duration_of_stay                   int64
pflegetage neu                    string
pccl                               int64
pccl neu                          string
old_pd                            string
primary_diagnosis                 string
added_icds                        string
removed_icds                      string
added_chops                       string
removed_chops                   

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,715177,33100584B601FADF,M,90,11,715177,C182,C182,M200,3,G18B,[D62],[],[990410::20180124],[]
1,716197,DEE16D56D71DD96A,W,72,14,716197,M4805,M4805,M200,3,I09C,[N184],[N183],[],[]
2,721128,34645F6C19043F5B,M,71,14,721128,K565,K565,M200,3,G21B,[G819],[],[],[]
3,721977,0098EFC426FD8F26,W,84,11,721977,S7201,S7201,M200,3,I46C,[T840],[],[],[]
4,725531,EBF3D9B44B52E53F,M,80,2,725531,I7022,I7022,M100,3,F59E,[G2090],[],[],[]
5,727730,1FECD162D99A5290,W,91,9,727730,J181,J181,M100,2,E77F,[N183],[N189],[],[]
6,727952,5415F800B9F4D0D6,M,88,4,727952,I5019,I5019,M100,3,F62C,[I420],[I517],[],[]
7,728588,F70905DAE9903086,W,84,10,728588,A481,A481,M100,3,E77C,[R64],[],[890A32],[]
8,731638,43A277B8C7954BC7,W,76,7,731638,M751,M751,M200,2,I29C,[J9610],[J9699],[],[]
9,737831,8FC3EAD412C59FE3,W,73,5,737831,J181,J181,M100,2,E77F,[G8253],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-14 09:37:01.357 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 6615 rows from the DB, for the hospital 'Hirslanden Linde' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,132158,727730,J181,"[I5019, J91, E1191, N183]",,[],W,91,0,0,9,0,01,2018-04-07,0,00,2018-04-16
1,135516,725531,I7022,"[E1140, Z921, G632, I672, Z866, ...]",395011:L:20180410,"[0046:L:20180410, 399010:L:20180410, 399031:L:20180410, 004B18:L:20180410, 0040:L:20180410]",M,80,0,0,2,0,01,2018-04-09,0,00,2018-04-11
2,135843,715177,C182,"[C772, K828, I495, Z4500, I1000, ...]",457311::20180123,"[512210::20180123, 4593::20180123, 009A13::20180123, 403X23::20180123, 990410::20180124]",M,90,0,0,11,0,01,2018-01-22,0,00,2018-02-02
3,136122,721128,K565,"[E789, I1190, N40, E86, N1793, ...]",5452:B:20180219,[9915::20180221],M,71,0,0,14,0,01,2018-02-19,0,00,2018-03-05
4,136230,731638,M751,"[M1991, M755, J4410, J9610]",818333:R:20180712,"[835X11:R:20180712, 808120:R:20180712, 831322:R:20180712, 802199:R:20180712, 808129:R:20180712]",W,76,0,0,7,0,01,2018-07-12,0,00,2018-07-19


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-14 09:37:01.809 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 10 cases ...
2022-11-14 09:37:02.787 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 10 cases into: 10 revisions, 75 diagnoses rows, 29 procedure rows


In [7]:
procedures_update

,aimedic_id,code,side,date,is_grouper_relevant,is_primary
0,135516,395011,L,2018-04-10,True,False
1,135516,0046,L,2018-04-10,False,False
2,135516,399010,L,2018-04-10,False,False
3,135516,399031,L,2018-04-10,False,False
4,135516,004B18,L,2018-04-10,False,False
5,135516,0040,L,2018-04-10,False,False
6,135843,457311,,2018-01-23,True,False
7,135843,512210,,2018-01-23,False,False
8,135843,4593,,2018-01-23,False,False
9,135843,009A13,,2018-01-23,False,False


In [8]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,132158,E77E,0.81,0.81,3,2022-12-31
1,135516,F59B,2.55,1.50,4,2022-12-31
2,135843,G18A,2.96,2.96,4,2022-12-31
3,136122,G04B,3.49,3.49,4,2022-12-31
4,136230,I27A,3.02,3.02,4,2022-12-31
5,136444,F62B,1.30,1.30,4,2022-12-31
6,136652,I06C,3.90,3.90,4,2022-12-31
7,136708,E77A,1.80,1.80,4,2022-12-31
8,137001,I46A,2.61,2.61,4,2022-12-31
9,137966,E77E,0.81,0.81,3,2022-12-31


In [9]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-14 09:37:02.813 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 10 cases into the 'Revisions' table ...
2022-11-14 09:37:03.685 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 10 rows from the "Revisions" table, which is about to be updated
2022-11-14 09:37:03.779 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 10 cases into the 'Revisions' table
2022-11-14 09:37:03.783 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 75 rows into the 'Diagnoses' table ...
2022-11-14 09:37:05.418 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 75 rows into the 'Diagnoses' table
2022-11-14 09:37:05.422 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 29 rows into the 'Procedures' table